In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import csv
import re

In [3]:
def clear_numeric_fields(file: str):
    r = csv.reader(open(file=file))
    lines = list(r)
    new_lines = []

    for line in lines:
        new_lines.append([re.sub(r'_$', '', el) for el in line])

    with open(file=file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerows(new_lines)


In [30]:
def input_pipeline(path: str) -> pd.DataFrame:

    #clear_numeric_fields(path)

    column_types = {"ID": str, #0
                    "Customer_ID": str, #0
                    "Month": str, #1
                    "Name": str, #2
                    "Age": np.int16, #3
                    "SSN": str, #4
                    "Occupation": str, #5
                    "Annual_Income": np.float32, #6
                    "Monthly_Inhand_Salary": np.float32, #7
                    "Num_Bank_Accounts": np.int16, #8
                    "Num_Credit_Card": np.int16, #9
                    "Interest_Rate": np.float32, #10
                    "Num_of_Loan": np.int16, #11
                    "Type_of_Loan": str, #12
                    "Delay_from_due_date": np.int16, #13
                    "Num_of_Delayed_Payment": np.float16, #14
                    "Changed_Credit_Limit": np.float32, #15
                    "Num_Credit_Inquiries": np.int16, #16
                    "Credit_Mix": str, #17
                    "Outstanding_Debt": np.float32, #18
                    "Credit_Utilization_Ratio": np.float32, #19
                    "Credit_History_Age": str, #20
                    "Payment_of_Min_Amount": str, #21
                    "Total_EMI_per_month": np.float32, #22
                    "Amount_invested_monthly": np.float32, #23
                    "Payment_Behaviour": str, #24
                    "Monthly_Balance": np.float32, #25
                    "Credit_Score": str} #26 dtype=column_types, 
    
    ofJedi = pd.read_csv(filepath_or_buffer=path, on_bad_lines="skip")
    ofJedi.fillna(value=0, axis=0, inplace=True)

    ofJedi["Age"] = ofJedi["Age"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Annual_Income"] = ofJedi["Annual_Income"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Num_of_Loan"] = ofJedi["Num_of_Loan"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Num_of_Delayed_Payment"] = ofJedi["Num_of_Delayed_Payment"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Amount_invested_monthly"] = ofJedi["Amount_invested_monthly"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Changed_Credit_Limit"] = ofJedi["Changed_Credit_Limit"].str.replace(pat=r"_*", repl="", regex=True)
    ofJedi["Changed_Credit_Limit"] = ofJedi["Changed_Credit_Limit"].str.replace(pat="0-", repl="-")
    ofJedi["Changed_Credit_Limit"] = ofJedi["Changed_Credit_Limit"].str.replace(pat="", repl="0")
    ofJedi["Monthly_Balance"] = ofJedi["Monthly_Balance"].str.replace(pat=r"_*", repl="", regex=True)

    ofJedi = ofJedi.astype(dtype=column_types)
    ofJedi.drop(columns=["ID", "Customer_ID", "Name", "SSN", "Month", "Occupation"], inplace=True)

    return ofJedi

In [5]:
def count_null_rows(df):
    for col in df.columns:
        na_sum = df[col].isna().sum()
        sum = len(df)
        
        if na_sum > 0:
            print("{0}:\t{1}".format(col, (na_sum/sum)))

In [6]:
def credit_history_age_clean(df):
    df.dropna(subset=["Credit_History_Age"], inplace=True)
    df["Credit_History_Age"] = df["Credit_History_Age"].str.slice(start=0, stop=2).astype(dtype=int)

In [7]:
def check_objects_per(df: pd.DataFrame) -> None:
    count_rows = len(df)
    for col in df.columns:
        if df[col].dtype == object:
            print(col)
            print(df[col].value_counts()/count_rows)
            print("\n")

In [8]:
def age_cleaner(df: pd.DataFrame, indexes: np.array) -> pd.DataFrame:
    pass

In [9]:
def age_analysis(x: pd.Series) -> np.array:
    print(x.head())
    print("Stardard Deviation:\t{0}\nMean:\t{1}\nVariance:\t{2}\nMaximum:\t{3}\nMinimum:\t{4}".format(x.std(), x.mean(), x.var(), x.max(), x.min()))

    print(x.describe())

    print("\nLength over 80:\t{0}".format(len(x[x > 80])))
    print("\nLength under 18:\t{0}".format(len(x[x < 18])))

    print("\nPercentage over 80:\t{0}".format(len(x[x > 80])/len(x)))
    print("\nPercentage under 18:\t{0}".format(len(x[x < 18])/len(x)))

    sns.boxplot(x=x)
    sns.displot(data=x, kind="kde")
    plt.show()

In [10]:
def generate_boxplot(df: pd.DataFrame) -> None:
    for col in df.columns:
        if df[col] != object:
            print(sns.boxplot(data=df[col]))

In [31]:
train_df = input_pipeline(path="train.csv")
train_df.head()

C:\Users\gugao\AppData\Local\Temp\ipykernel_4732\614064891.py:34: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  ofJedi = pd.read_csv(filepath_or_buffer=path, on_bad_lines="skip")


ValueError: could not convert string to float: '0-020.00010': Error while type casting for column 'Changed_Credit_Limit'

In [ ]:
for col in train_df.columns:
    print("{0}:\t{1}".format(col, train_df[col].dtype))

In [ ]:
for col in train_df.columns:
    print("{0}:\t{1}".format(col, train_df[col].isna().sum()))

In [ ]:
len(train_df)

In [ ]:
for col in train_df.columns:
    na_sum = train_df[col].isna().sum()
    sum = len(train_df)

    if na_sum > 0 and train_df[col].dtype==object:
        print("{0}:\t{1}".format(col, (na_sum/sum)))

In [ ]:
train_df[train_df["Type_of_Loan"].isna() & train_df["Credit_History_Age"].isna()]

Type_of_Loan is too sparse, and too many nan values

In [ ]:
train_df["Type_of_Loan"].value_counts()

In [ ]:
train_df.drop(columns=["Type_of_Loan"], inplace=True)
train_df

In [ ]:
count_null_rows(df=train_df)

In [ ]:
train_df["Credit_History_Age"].value_counts()

In [ ]:
credit_history_age_clean(train_df)
train_df

In [ ]:
count_null_rows(train_df)

In [ ]:
train_df.dropna(inplace=True)
train_df

In [ ]:
for col in train_df.columns:
    if train_df[col].dtype == object:
        print(col)
        print(train_df[col].value_counts())
        print("\n")

In [ ]:
train_df[col].value_counts()

In [ ]:
train_df["Credit_Score"].value_counts()/2

In [ ]:
check_objects_per(df=train_df)

In [ ]:
train_df.drop(index=train_df[train_df.Payment_Behaviour == "!@9#%8"].index, inplace=True)

In [ ]:
train_df.drop(index=train_df[train_df.Credit_Mix == "_"].index, inplace=True)

In [ ]:
check_objects_per(df=train_df)

In [ ]:
len(train_df)

In [ ]:
train_df.columns

In [ ]:
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
age_analysis(train_df["Age"])

In [ ]:
corrMatrix = train_df.corr()
sns.heatmap(corrMatrix, annot=True)
# plt.figure(figsize=(20, 100))
plt.show()

In [ ]:
selected_col = train_df.loc[:, ["AnnualIncome", "Monthly_Inhand_Salary", "Num_of_Loan", "Credit_Score"]]
selected_col = selected_col.corr()
sns.heatmap(selected_col, annot=True)
# plt.figure(figsize=(20, 100))
plt.show()

In [ ]:
train_df.groupby(by=["Credit_Score"]).agg({"Age": "mean"})